In [2]:
import gym

In [3]:
env = gym.make('SpaceInvaders-v0')

In [4]:
env.reset()
env.render()

True

In [6]:
for t in range(10000):
    env.render()
    ac =env.action_space.sample()
    o,r,d,i = env.step(ac)
    if d:
        env.reset()
env.close()

In [9]:
# con la funzione action_space riusciamo a trovare lo spazio delle azioni 
print (env.action_space)

Discrete(6)


In [10]:
# con la funzione observation_space, invece, troviamo lo spazio delle osservazioni, in questo caso le dimensioni in pixel della
# nostra simulazione
print (env.observation_space)

Box(210, 160, 3)


In [18]:
print (env.action_space)

Discrete(6)
